In [2]:
import numpy as np
import xlrd as pxl
import os
import glob
from image_util import *

In [3]:
# generate datatset
input_edge = 256

for name in glob.glob("./data/PapSmearDatasetSingleCellNoSegmented/*/*"):
    pth = name.split('/')
    img = cv.imread(name)
    aug = augment_images(img, input_edge)
    pth[2] = 'PapSmearDatasetSingleCellAugmented'
    fl = pth[-1].split('.')
    name, ext = fl[0], fl[1]
    ext = '.' + ext
    for i in range(len(aug)):
        pth[-1] = name + '-aug' + str(i) + ext
        new_pth = '/'.join(pth)
        if not os.path.exists(os.path.dirname(new_pth)):
            os.makedirs(os.path.dirname(new_pth))
        cv.imwrite(new_pth, aug[i])

In [4]:
# read data - ONLY RUN ONCE
# load regressed nucleus/citoplasm ratios
def make_id(fname):
    return ''.join(fname.split('-')[:2])


valuesDict = dict()
sheet = pxl.open_workbook('./data/PapSmearDatasetSingleCell/PapSmearDatasetAnalysis.xls').sheet_by_index(0)
val_n = len(sheet.col(0))

for i in range(1, val_n):
    valuesDict[make_id(sheet.cell_value(i, 0))] = sheet.cell_value(i, 3)


    
#load images
Xall = list() 
Yall = list() 

for pth in glob.glob("./data/PapSmearDatasetSingleCellAugmented/*/*"):
    img = cv.imread(pth)
    Id = make_id(os.path.basename(pth))
    Xall.append(img)
    Yall.append(valuesDict[Id])

Xall = np.array(Xall)
Yall = np.array(Yall)

In [5]:
#shuffle data - ONLY RUN ONCE
rand_perm = np.random.permutation(Xall.shape[0])
Xall = Xall[rand_perm]
Yall = Yall[rand_perm]

In [6]:
# split train and test - ONLY RUN ONCE
test_ratio = 0.2
cut = int(Xall.shape[0]*(1.-test_ratio))
Xtrain = Xall[:cut]
Xtest = Xall[cut:]
Ytrain = Yall[:cut]
Ytest = Yall[cut:]

In [7]:
#save data - ONLY RUN ONCE
np.save('./data/Xtrain', Xtrain)
np.save('./data/Xtest', Xtest)
np.save('./data/Ytrain', Ytrain)
np.save('./data/Ytest', Ytest)